# Airflow

Airflow is a tool that allows to schedule a set of processes commonly used in ETL pipelines and sometimes in ML automation. This section considers typical ways to use Airflow.

We typically are typically run airflow in the Docker container to make sure that we are working in a clean environment. Use following command to run the container.

Build the docker image described in the `airflow_files/dockerfile` and run it with the `standalone` command.

```bash
docker build -f packages/airflow_files/dockerfile .
docker run -d --rm --name airflow -p 8080:8080 -v ./:/knowledge airflow standalone
```

Image that is used as an example configured in such way to create default user with login `user` and password `user` that will be used as credentials for the airflow server.

## Configuration file

The global configuration of the airflow is stored in the special file: `airflow.cfg`. Different installations put this file in different places (as usual). Typical locations are `/opt/airflow/airflow.cfg` and `~/airflow/airflow.cfg`.

---

Any way use following command to find the location of the `airflow.cfg` on your disk.

In [9]:
!find / -name airflow.cfg

/opt/airflow/airflow.cfg


## Adding a dag

This section shows the minimal actions needed to create an airflow dag. It is based on the [Fundamental conceptse official tutorial](https://airflow.apache.org/docs/apache-airflow/stable/tutorial/fundamentals.html), but shows only the minimal command to add a dag and verify that it works.

---

First you need to identify the folder containing dags. This folder is specified by the `dogs_folder` parameter of the configuration file.

In [1]:
!cat /opt/airflow/airflow.cfg | grep dags_folder

dags_folder = /opt/airflow/dags


DAG must be implemented by the special DAG file. This is the file that contains `airflow.models.DAG` object. It takes a lot of settings that determine it's behavior but in general it needs only `dag_id` to be specified. The following cell defines a dag with id `tutorial`.

In [2]:
%%writefile /opt/airflow/dags/tutorial.py
from airflow.models.dag import DAG

with DAG("tutorial") as dag:
    pass

Writing /opt/airflow/dags/tutorial.py


The following command causes airflow to add DAG to its databases.

In [3]:
!airflow db migrate

DB: sqlite:////opt/airflow/airflow.db
Performing upgrade to the metadata database sqlite:////opt/airflow/airflow.db
[2025-03-22T14:14:21.698+0000] {migration.py:207} INFO - Context impl SQLiteImpl.
[2025-03-22T14:14:21.700+0000] {migration.py:210} INFO - Will assume non-transactional DDL.
[2025-03-22T14:14:21.704+0000] {migration.py:207} INFO - Context impl SQLiteImpl.
[2025-03-22T14:14:21.704+0000] {migration.py:210} INFO - Will assume non-transactional DDL.
[2025-03-22T14:14:21.705+0000] {db.py:1675} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Database migrating done!


With `airflow dags list` you can show DAGs that are seen by the airflow.

In [4]:
!airflow dags list

dag_id   | fileloc                       | owners | is_paused
=========+===============================+========+==========
tutorial | /opt/airflow/dags/tutorial.py |        | True     
                                                             


As a result there is a dag we have added below.

## Operators & tasks

**Operators** in Airflow are templates that define a common way of performing a task.  

A **task** is a concrete instance of an operator.

Here is the list of the common airflow operators:

| Operator          | Description |
|------------------|-------------|
| `PythonOperator`  | Executes a Python function. |
| `BashOperator`    | Runs a Bash command or script. |
| `PostgresOperator` | Executes a SQL query in a PostgreSQL database. |
| `MySqlOperator`   | Executes a SQL query in a MySQL database. |
| `SqliteOperator`  | Executes a SQL query in a SQLite database. |
| `MsSqlOperator`   | Executes a SQL query in an MS SQL Server database. |
| `SnowflakeOperator` | Executes a query in Snowflake. |
| `BigQueryOperator` | Executes a SQL query in Google BigQuery. |
| `HiveOperator`    | Executes a HiveQL query in Apache Hive. |
| `S3FileTransformOperator` | Transfers and transforms files in AWS S3. |
| `EmailOperator`   | Sends an email. |
| `HttpOperator`    | Sends an HTTP request. |
| `DummyOperator` (Deprecated: Use `EmptyOperator`) | A placeholder operator that does nothing. |
| `EmptyOperator`   | A lightweight no-op operator, replacing `DummyOperator`. |
| `BranchPythonOperator` | Chooses the next task dynamically based on a Python function. |
| `TriggerDagRunOperator` | Triggers another DAG. |
| `KubernetesPodOperator` | Runs a task inside a Kubernetes pod. |

---

As example consider basic tools to handle the tasks in mlflow.

The following code defines DAG with `bash` task and loads it to the airflow.

In [5]:
%%writefile /opt/airflow/dags/tutorial.py
from airflow.models.dag import DAG
from airflow.operators.bash import BashOperator

with DAG("tutorial") as dag:
    BashOperator(
        task_id="hello_world",
        bash_command="echo 'Hello world'"
    )

Overwriting /opt/airflow/dags/tutorial.py


In [8]:
!airflow db migrate &> /dev/null

List the tasks of the DAG with `airflow tasks list <name of the dug>`.

In [9]:
!airflow tasks list tutorial

hello_world


Run tasks for testing purposes using `airflow tasks test <name of dag> <name of task>`

In [13]:
!airflow tasks test tutorial hello_world

[2025-03-22T14:47:49.566+0000] {dagbag.py:588} INFO - Filling up the DagBag from /opt/airflow/dags
[2025-03-22T14:47:49.698+0000] {taskinstance.py:2614} INFO - Dependencies all met for dep_context=non-requeueable deps ti=<TaskInstance: tutorial.hello_world __airflow_temporary_run_2025-03-22T14:47:49.584901+00:00__ [None]>
[2025-03-22T14:47:49.702+0000] {taskinstance.py:2614} INFO - Dependencies all met for dep_context=requeueable deps ti=<TaskInstance: tutorial.hello_world __airflow_temporary_run_2025-03-22T14:47:49.584901+00:00__ [None]>
[2025-03-22T14:47:49.702+0000] {taskinstance.py:2867} INFO - Starting attempt 0 of 1
[2025-03-22T14:47:49.703+0000] {taskinstance.py:2948} WARNING - cannot record queued_duration for task hello_world because previous state change time has not been saved
[2025-03-22T14:47:49.703+0000] {taskinstance.py:2890} INFO - Executing <Task(BashOperator): hello_world> on 2025-03-22 14:47:49.584873+00:00
[2025-03-22T14:47:50.022+0000] {taskinstance.py:3134} INFO -

The outputs contain a lot of logging information, but now only the line after "Output" is important. The next cell highlights it.

In [14]:
!airflow tasks test tutorial hello_world | grep "Output" -A 1

[2025-03-22T14:48:00.890+0000] {subprocess.py:99} INFO - Output:
[2025-03-22T14:48:00.891+0000] {subprocess.py:106} INFO - Hello world
